In [1]:
#Import necessary packages
import numpy as np
from scipy.integrate import quad
from scipy import stats
import warnings
warnings.simplefilter('ignore')

#Install import_ipynb so we can read dependenies as jupyter notebooks
!pip install ipynb

#Import jupyer notebook depdendencies 
from ipynb.fs.full.FFT_option_valuation_LEWIS import BSM_call_value_FFT, H93_call_value_FFT
from ipynb.fs.full.Lewis_Integration_option_valuation import BSM_call_value, H93_call_value


In [2]:
#BSM analytical formula

def BSM_call_valueAnalytical(S0, K, T, r, d, sigma):
    ''' Valuation of European call option in BSM Model.
    --> Analytical Formula.
    Parameters
    ==========
    S0: float
    initial stock/index level
    K: float
    strike price
    T: float
    time-to-maturity (for t=0)
    r: float
    constant risk-free short rate
    d: float
    dividend yield
    sigma: float
    volatility factor in diffusion term
    Returns
    =======
    call_value: float
    European call option present value
    '''
    d1 = (np.log(S0 / K) + (r-d + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S0 / K) + (r-d - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    BS_C = (S0*np.exp(-d*T)* stats.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * stats.norm.cdf(d2, 0.0, 1.0))
    return BS_C


In [4]:
# BSM Parameters
S0 = 100.0
K = 100.0
T = 1.
r = 0.05
sigma = 0.2
d = 0

In [5]:
BSM_call_value(S0, K, T, r, d, sigma)

10.450583572184826

In [6]:
BSM_call_value_FFT(S0, K, T, r, d, sigma)

10.450813049892247

In [7]:
BSM_call_valueAnalytical(S0, K, T, r, d, sigma)

10.450583572185565

In [8]:
import numpy.random as npr
np.random.seed(1500)

def H93_call_value_MC(S0, strike, T, v0, kappa, theta, sigma, rho, r, d, M, I):
    ''' Price of European Call option under Heston model (Montecarlo Simulation)
    Parameters
    ==========
    S0 :     float, initial stock value
    strike : float,  strike price
    T:       float,  maturity date (in year fractions)
    v0 :     float, initial instantaneous variance value
    kappa :  float, mean-reversion rate
    theta :  float, long-run average of the variance
    sigma :  float, vol-vol
    rho :    float, instanteneous correlation of Wiener processes
    r :      float, constant risk-free short rate
    d :      float, dividend yield
    M:       int,   number of time steps
    I:       int,   number of simulation pathes
    
    Returns
    =======
    C0: float, Price European Call
    '''
        
    #cho_mat
    corr_mat = np.zeros((2, 2))
    corr_mat[0, :] = [1.0, rho]
    corr_mat[1, :] = [rho, 1.0]
    cho_mat = np.linalg.cholesky(corr_mat)
    
    ran_num = npr.standard_normal((2, M + 1, I)) 
    dt = T / M
    
    # row represents each path, column represents time epoches
    # different from the lecture note where we used row for time, column for pathes
    vh = np.zeros((I, M+1)) 
    vh[:, 0] = v0
    S = np.zeros((I, M+1)) 
    S[:, 0] = S0
    
    for t in range(1, M + 1):
        ran = np.dot(cho_mat, ran_num[:, t, :])
        vh[:, t] = (vh[:, t - 1] + kappa * (theta - np.maximum(vh[:, t - 1], 0)) * dt
          + sigma * np.sqrt(np.maximum(vh[:, t - 1], 0)) * np.sqrt(dt)  
          * ran[1])
        vh[:, t] = np.maximum(0, vh[:,t])
        S[:, t] = S[:, t - 1] * np.exp((r - d - 0.5 * vh[:, t]) * dt + np.sqrt(vh[:, t]) * ran[0] * np.sqrt(dt))
    
    ST = S[:, -1]
    hT = np.maximum(ST - strike, 0)
    # calculate MCS estimator
    C0 = np.exp(-(r-d) * T) * np.mean(hT)
    return C0

In [9]:
# Heston Parameters
S0 = 100.0
K = 100.0
T = 1.0
r = 0.05
kappa_v = 2
theta_v = 0.01
sigma_v = 0.1
rho = -0.5
v0 = 0.01
d = 0

In [10]:
H93_call_value(S0, K, T, r, d, kappa_v, theta_v, sigma_v, rho, v0)

6.874940187021394

In [11]:
H93_call_value_FFT(S0, K, T, r, d, kappa_v, theta_v, sigma_v, rho, v0)

6.87516966472441

In [12]:
M = 250
I = 500000
H93_call_value_MC(S0, K, T, v0, kappa_v, theta_v, sigma_v, rho, r, d, M, I)

5.179618683296158